# Processing scraped jobs

Once job details and descriptions are scraped a few more things need to happen:

* compare today's vs. yesterday's jobs
* when new job comes in, add day tracker (days = 0)
* if job not new, days += 1

**Import some libraries**

In [42]:
import pandas as pd
import numpy as np

# plotting pacakges
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
%matplotlib inline

# ignore matplotlib/seaborn deprecation warning... it just looks ugly
import warnings
warnings.filterwarnings("ignore", message="The 'normed' kwarg is deprecated, and has been ")

# assortment of other libraries
import re
import time
from nltk.corpus import stopwords
import string
from collections import Counter
import pandas as pd
import numpy as np
import arrow

In [20]:
df1 = pd.DataFrame({"test1": [3,2,1,3,1], "test2": [6,7,4,2,1]})
df2 = pd.DataFrame({"test1": [1,5,3,3,2]})
print(df1)
df3 = df2.merge(df1, on=['test1'], how="right", indicator=True)

df3.loc[df3['test2'].isnull(), 'test2'] = 0
big = df3[df3['test2']>4].index
add = [1,-5,0,1,5]
df1['test2'] += add
print(df1)
df1 = df1.sort_values(by=['test2'], ascending=False)
print(df1)
print(df1.iloc[:100])

   test1  test2
0      3      6
1      2      7
2      1      4
3      3      2
4      1      1
   test1  test2
0      3      7
1      2      2
2      1      4
3      3      3
4      1      6
   test1  test2
0      3      7
4      1      6
2      1      4
3      3      3
1      2      2
   test1  test2
0      3      7
4      1      6
2      1      4
3      3      3
1      2      2


***
***
***

# INITIALIZE FIRST DF

In [43]:
df_test = pd.read_csv("./data/scrapes/raw_scrapes/glassdoor-df-06-06-2018.csv", encoding="ISO-8859-1")
df_test2 = pd.read_csv("./data/scrapes/raw_scrapes/glassdoor-df-06-08-2018.csv", encoding="ISO-8859-1")

In [44]:
def initialize_first_df(df):

    # get rid of duplicates
    dups = df.duplicated(subset=['position','company','location','description'])
    df = df[~dups]

    # add some new columns of interest
    df['days_posted'] = 0
    df['date_posted'] = arrow.now().format('MM-DD-YYYY')
    df['applied'] = "No"
    df['date_applied'] = np.nan
    df['strikes'] = 0
    df['rank'] = 0
    
    return df

In [45]:
df_test = initialize_first_df(df_test)

c:\users\skarb\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\skarb\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\skarb\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [46]:
df_test.head(n=4)

,position,company,location,description,link,days_posted,date_posted,applied,date_applied,strikes,rank
0,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...,0,06-11-2018,No,NaN,0,0
1,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...,0,06-11-2018,No,NaN,0,0
2,Machine Learning Scientist (University Grad),Electronic Arts,"Redwood City, CA",We are EA\r\n\r\nAnd we make games how cool is...,https://www.glassdoor.com/partner/jobListing.h...,0,06-11-2018,No,NaN,0,0
3,New Grad - Data Scientist,Viasat,"Boston, MA",Job ResponsibilitiesBecome a personal contribu...,https://www.glassdoor.com/partner/jobListing.h...,0,06-11-2018,No,NaN,0,0


In [ ]:
def filter_by_location(df):

    filter_index = []
    locations = df['location'].tolist()

    for location in locations:
        try:
            state = location.split(',')[1].strip()
            if state in self.locations_of_interest:
                filter_index.append(True)
            else:
                filter_index.append(False)
        except:
            if location == 'Remote':
                filter_index.append(True)
            else:
                filter_index.append(False)

    return(df[filter_index])

def filter_by_position_title(self):

    filter_jobs = []
    jobs = self.df_new['position'].tolist()

    for job in jobs:
        job_words = job.split(" ")
        for word in job_words:
            if word.lower() in filter_words:
                rank = 2
                break;
            else:
                include = True
        if include:
            filter_jobs.append(True)
        else:
            filter_jobs.append(False)
    df = df[filter_jobs]

def filter_by_description(self):
    pass

In [47]:
# df_test = filter_by_location(df_test)
# df_test = filter_by_position_title(df_test)
# df_test = filter_by_description(df_test)
df_test.to_csv("./data/scrapes/cleaned_scrapes/glassdoor-df-06-06-2018.csv", index=False)

***
***
***

# PROCESSOR CLASS

In [25]:
import pandas as pd
import numpy as np
import arrow
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import re
from collections import Counter
import string
import time
import sys


class process_raw_scrape():
    
    def __init__(self, old_df, df_new):
    
        self.df_new = df_new
        self.df_old = old_df
        self.archive_df = pd.DataFrame()
        
        # used for ranking system
        self.locations_of_interest = ['ca', 'co', 'hi', 'me', 'mi', 'ny', 'oh', 'or', 'tx', 'wa'] # +2
        self.words_of_interest = ['data'] # +1
        self.filter_words = ['senior', 'sr.', 'sr', 'director', 'manager', 'research', 'architect'] # -1
        self.key_skills = ['data science', 'python', 'statistical analysis']

    # DONE #
    def combine_dfs(self):
        
        start = time.time()
        
        print("Removing duplicates from today's web scrape.")
        #prepare df_new
        dups = self.df_new.duplicated(subset=['position','company','location','description'])
        self.df_new = self.df_new[~dups]

        print("Merging today's web scrape with yesterday's.")
        # full outer join on old and new df (and get rid of duplicate columns)
        self.df_new = self.df_new.merge(self.df_old, on=['company','position','location','description'], how="outer", indicator=True)
        self.df_new = self.df_new.drop(columns=['link_x'])
        self.df_new = self.df_new.rename(columns= {'link_y':'link'})
        self.df_new.loc[self.df_new['strikes'].isnull(), 'strikes'] = 0

        # mark any aging jobs (but don't remove just yet)
        self.df_new.loc[self.df_new['_merge']=='right_only', 'strikes'] += 1
        
        # save old jobs to archive
        self.archive_df = self.df_new[self.df_new['strikes'] >= 3]
        if not self.archive_df.empty:
            self.df_new = self.df_new.drop(self.archive_df.index)
            self.archive_df.to_csv("../data/scrapes/archived_scrapes/glassdoor-df-{}.csv".format(arrow.now().format('MM-DD-YYYY')).csv, 
                                   index=False)
        else:
            print("No jobs old enough to archive today.")

        # days_posted
        self.df_new.loc[self.df_new['_merge']=='both', 'days_posted'] += 1
        self.df_new.loc[self.df_new['days_posted'].isnull(), 'days_posted'] = 0 

        # date_posted
        self.df_new.loc[self.df_new['date_posted'].isnull(), 'date_posted'] = arrow.now().format('MM-DD-YYYY') 

        # applied
        self.df_new.loc[self.df_new['applied'].isnull(), 'applied'] = "No"   
        
        # rank
        self.df_new.loc[self.df_new['rank'].isnull(), 'rank'] = 0

        # drop merge
        self.df_new = self.df_new.drop(columns=['_merge'])
        
        save_location = "../data/scrapes/cleaned_scrapes/glassdoor-df-{}.csv".format(arrow.now().format('MM-DD-YYYY'))
        self.df_new.to_csv(save_location, index=False)
        print("Cleaned web scrape saved to {}".format(save_location))
        
        end = time.time()
        
        print("Merging dataframes complete!")
        print("Total time: {:.2f}".format((end-start)/60))
        print("\n")

    # DONE #
    def filter_by_location(self):

        locations = self.df_new['location'].tolist()
        ranks = [0]*len(locations)
        
        for i,location in enumerate(locations):
            try:
                if location == 'Remote':
                    ranks[i] += 3
                    continue;
                state = location.split(',')[1].strip()
                if state.lower() in self.locations_of_interest:
                    ranks[i] += 3
            except:
                pass

        self.df_new['rank'] += ranks

    # DONE #
    def filter_by_position_title(self):

        jobs = self.df_new['position'].tolist()
        ranks = [0]*len(jobs)
        
        for i,job in enumerate(jobs):
            
            job = job.lower()
            
            if (re.search("scientist", job)) and not (re.search("data", job)):
                ranks[i] += -3
            
            first_plus = True
            first_minus = True
            
            job_words = job.split(" ")
            for word in job_words:
                if (first_plus) and (word in self.words_of_interest):
                    first_plus = False
                    ranks[i] += 1
                elif (first_minus) and (word in self.filter_words):
                    first_minus = False
                    ranks[i] += -1
                
                
        self.df_new['rank'] += ranks
    
    # DONE #
    def filter_by_description(self):
        
        descriptions = self.df_new['descriptions'].tolist()
        ranks = [0]*len(descriptions)
        
        for i,description in enumerate(descriptions):
            
            lowered = description.lower()
            bachelor = True if re.search("bachelor|b\.?s\.?", lowered) else False
            master = True if re.search("master|ms|m\.s\.|ms\.", lowered) else False
            phd = True if re.search("doctorate|p\.?h\.?d\.?", lowered) else False
            experience_2 = True if re.search("2\+?year[s]?", lowered) else False
            experience_5 = True if re.search("5\+?year[s]?", lowered) else False
            experience_10 = True if re.search("10\+?year[s]?", lowered) else False
            good_words = True if re.search(words_of_interest, lowered) else False

            if (experience_5 and phd) or (phd and not master) or (experience_10):
                ranks[i] += -2
            elif (experience_2 and not phd and not master) or (experience_5 and not phd):
                ranks[i] += -1
            elif (master and not phd and not experience_10) or (bachelor):
                ranks[i] += 2
            if good_words:
                ranks[i] += 2
            
        self.df_new['rank'] += ranks
    
    # DONE #
    def save_top_jobs(self):
        
        start = time.time()
        
        print("Beginning to ")
        top_jobs = Counter(self.df_new['position'].tolist()).most_common(20)
        
        save_location = '../data/app_data/current_plot_data/top-jobs-{}.txt'.format(arrow.now().format('MM-DD-YYYY'))
        with open(save_location, 'w') as file:
            for job in top_jobs:
                if isinstance(job[0], float):
                    continue;
                elif (len(job[0]) > 45):
                    continue;
                else:
                    file.write('{0};{1}'.format(job[0], job[1]))
                    file.write('\n')
       
        end = time.time()
        print("Finished parsing top jobs. Saved to {}".format(save_location))
        print("Total time: {:.2f} minutes.".format((end-start)/60))
        print("\n")
    
    # DONE #
    def save_top_terms(self):
        
        start = time.time()
        
        print("Beginning to parse descriptions for top terms.")
        descriptions = self.df_new['description'].tolist()
        stop_words = set(stopwords.words('english'))
        common_terms = []
        
        for description in descriptions:
            
            words = tester.strip().replace('\n', ' ').translate(string.punctuation) # strip down to words
            words = re.sub(r'[^\w\s]', '', words)

            tokens = word_tokenize(words)
            tokens = [word for word in tokens if not word in stop_words]

            unigrams = list(ngrams(tokens, 1))
            bigrams = list(ngrams(tokens, 2))
            trigrams = list(ngrams(tokens, 3))

            for unigram in unigrams:
                common_terms.append(unigram[0])

            for bigram in bigrams:
                common_terms.append("{0} {1}".format(bigram[0], bigram[1]))

            for trigram in trigrams:
                common_terms.append("{0} {1} {2}".format(trigram[0], trigram[1], trigram[2]))

                
        top_terms = Counter(common_terms).most_common(20)

        save_location = "../data/app_data/current_plot_data/top-terms-{}.txt".format(arrow.now().format('MM-DD-YYYY'))
        with open(save_location, 'w') as file:
            for word in counted_words:
                try:
                    file.write("{0};{1}".format(word[0], word[1]))
                    file.write("\n")
                except:
                    print("Couldn't write top term: '{0}' ({1}) to file".format(word[0], type(word[0])))
        end = time.time()
        print("Finished parsing top terms. Saved to {}".format(save_location))
        print("Total time: {:.2f} minutes".format((end-start)/60))
        print("\n")
        
    # DONE #  
    def save_df(self):
        
        # save cleaned_scrape to disk
        self.df_new.to_csv("../data/scrapes/clean_filtered_scrapes/glassdoor-df-{}.csv".format(arrow.now().format('MM-DD-YYYY')), index=False)
        print("Saved cleaned and filtered web scrape.")
        
        # save cleaned_scrape without descriptions for app upload
        jobs_for_app = self.df_new.drop('description', axis=1)
        jobs_for_app.to_csv("../data/app_data/jobs-for-app.csv", index=False)
        print("Saved jobs for web app.")
        
    
# main program to run from terminal/command-line
if __name__ == '__main__':
    
    # get today's and yesterday's web scrapes ready
    old_df = pd.read_csv("../data/scrapes/cleaned_scrapes/glassdoor-df-{}.csv".format(arrow.now().shift(days=-1).format('MM-DD-YYYY')), 
                         encoding="ISO-8859-1")
    new_df = pd.read_csv("../data/scrapes/raw_scrapes/glassdoor-df-{}.csv".format(arrow.now().format('MM-DD-YYYY')), 
                         encoding="ISO-8859-1")
   

    ######################################
    # STEP 1: initialize processor class #
    ######################################
    try:
        job_processor = process_raw_scrape(old_df, new_df)
    except:
        print("Issue initializing processor class. Shutting down.")
        sys.exit()
        
        
    ######################################
    # STEP 2: compare yesterday to today #
    ######################################
    try:
        job_processor.combine_dfs()
    except:
        print("Issue in combine_dfs(). Shutting down.")
        sys.exit()

        
    ###################################################
    # STEP 3: set rankings and remove irrelevant jobs #
    ###################################################
    try:
        job_processor.filter_by_location()
    except:
        print("Issue in filter_by_location(). Shutting down.")
        sys.exit()
    try:
        job_processor.filter_by_position_title()
    except:
        print("Issue in filter_by_position_title(). Shutting down.")
        sys.exit()
    try:
        job_processor.filter_by_description()
    except:
        print("Issue in filter_by_description(). Shutting down.")
        sys.exit()
           
            
    #########################
    # STEP 4: get plot data #
    #########################
    try:
        job_processor.save_top_jobs()
    except:
        print("Issue in save_top_jobs(). Shutting down.")
        sys.exit()
        
    try:
        job_processor.save_top_terms()
    except:
        print("Issue in save_top_terms(). Shutting down.")
        sys.exit()
    
    
    ###########################
    # STEP 5: save cleaned df #
    ###########################
    try:
        job_processor.save_df()
    except:
        print("Issue in save_df(). Shutting down.")
        sys.exit()
    
    # we made it to the end!
    print("RAW WEB SCRAPE FULLY PROCESSED!")


In [49]:
practice = ["Scientist bio", "Data Scientist"]

for title in practice:
    title = title.lower()
    
    if (re.search("scientist", title)) and not (re.search("data", title)):
        print("booo", title)
    else:
        print("Yaaay", title)

booo scientist bio
Yaaay data scientist


In [21]:
test = [0]*20
print(test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [50]:
df_test.shape

(4906, 11)

In [54]:
test = df_test['description'].isnull().tolist()

len(test)

4906

In [60]:
df_test.shape

(4906, 11)

In [61]:
df_test[df_test['link'].isnull()].shape

(920, 11)

In [65]:
df_test[~df_test['link'].isnull()].shape

(3986, 11)

***
***
***

**Filter by words in description**

Main ideas:

1) Rank jobs 1-3 (1 highest).


2) Filter on some ideas.

* if 5+ years experience: rank 3
* if Phd and not Master's: rank 3
* if no mention of education word but experience 2+: rank 2
* if Phd and master's: rank 2
* if master's and not phd: rank 1
* if bachelor and/or master: rank 1

In [39]:
test_strings = ["Responsibilities include Phd or Master's", "experience require P.hd.", 
                "Miss Must like a master", "do you have masters or P.hd?",
                "5+ years experience with a phd in masters", "2 year experience in something",
                "5 year experience with master's and phd.", "requires M.S. or Phd. data science", "I have a master's data",
                "I can python your data!", "I'm a scientist.", "I'm a data!", "I can science ok"]

In [41]:
ranks = [0]*len(test_strings)
words_of_interest = "python|data science|data scientist|quantitative|analytical"

for i,test_string in enumerate(test_strings):
    lowered = test_string.lower()
    bachelor = True if re.search("bachelor|b\.?s\.?", lowered) else False
    master = True if re.search("master|ms|m\.s\.|ms\.", lowered) else False
    phd = True if re.search("doctorate|p\.?h\.?d\.?", lowered) else False
    experience_2 = True if re.search("2\+?year[s]?", lowered) else False
    experience_5 = True if re.search("5\+?year[s]?", lowered) else False
    experience_10 = True if re.search("10\+?year[s]?", lowered) else False
    good_words = True if re.search(words_of_interest, lowered) else False
    
    if (experience_5 and phd) or (phd and not master) or (experience_10):
        ranks[i] += -2
    elif (experience_2 and not phd and not master) or (experience_5 and not phd):
        ranks[i] += -1
    elif (master and not phd and not experience_10) or (bachelor):
        ranks[i] += 2
    if good_words:
        ranks[i] += 2
        
    print(lowered)
    print(ranks[i])
    print(bachelor,master,phd,experience_2,experience_5,experience_10,good_words)
    print("\n")
        
ranks

responsibilities include phd or master's
0
False True True False False False False


experience require p.hd.
-2
False False True False False False False


miss must like a master
2
False True False False False False False


do you have masters or p.hd?
0
False True True False False False False


5+ years experience with a phd in masters
0
False True True False False False False


2 year experience in something
0
False False False False False False False


5 year experience with master's and phd.
0
False True True False False False False


requires m.s. or phd. data science
2
False True True False False False True


i have a master's data
2
False True False False False False False


i can python your data!
2
False False False False False False True


i'm a scientist.
0
False False False False False False False


i'm a data!
0
False False False False False False False


i can science ok
0
False False False False False False False




[0, -2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0]

In [70]:
test = ['a','a','c','b','c','a']

counted = Counter(test).most_common()

for count in counted:
    print(count[0], count[1])

a 3
c 2
b 1


In [71]:
test = ('hi', 'there')

test[0]

'hi'

In [406]:
trigrams[0]

('Responsibilities', 'include', 'Phd')

In [410]:
common_terms

['Responsibilities',
 'include',
 'Phd',
 'Masters',
 'Responsibilities include',
 'include Phd',
 'Phd Masters',
 'Responsibilities include Phd',
 'include Phd Masters']

[('Masters', 1),
 ('include', 1),
 ('Phd Masters', 1),
 ('Responsibilities include Phd', 1),
 ('include Phd', 1),
 ('Responsibilities include', 1),
 ('Phd', 1),
 ('include Phd Masters', 1),
 ('Responsibilities', 1)]

In [260]:
ranks = []

for string in test_strings:
    lowered = string.lower()
    bachelor = True if re.search("bachelor|b\.?s\.?", lowered) else False
    master = True if re.search("master|ms|m\.s\.|ms\.", lowered) else False
    phd = True if re.search("doctorate|p\.?h\.?d\.?", lowered) else False
    experience_2 = True if re.search("2\+?year[s]?", lowered) else False
    experience_5 = True if re.search("5\+?year[s]?", lowered) else False
    experience_10 = True if re.search("10\+?year[s]?", lowered) else False
    
    if (experience_5 and phd) or (phd and not master) or (experience_10):
        ranks.append(3)
    elif (experience_2 and not phd and not master) or (experience_5 and not master):
        ranks.append(2)
    elif (master and not phd) or (bachelor) or ():
        ranks.append(1)
    else:
        ranks.append("?")
        
print(ranks)
    


['?', 3, 1, '?', '?', '?', '?', '?', 1]


**Parse the job descriptions for key word counts**

In [94]:
df_test = compare_days(df_old, df_new)

('YapStone', 'Data Scientist')
('eBay Inc.', 'Sr. Data Scientist')
('Reali', 'Product Manager')
('eBay Inc.', 'Data Scientist, Marketing')
('Esri', 'Data Scientist')
('Axia Technologies', 'Senior Product Manager')
('Trianz', 'Python Full Stack Developer')
('realtor.com', 'Sr / Staff Software Engineer')
('Entytle', '\r\n\r\n                        Data Analyst\r\n')
('Match', 'Growth Marketer (Growth Hacker)')
('Ancestry', 'Quality Engineer - Data Science Engineering Team')
('Esri', 'Data Science Product Engineer - ArcGIS API for Python')
('eBay Inc.', 'Staff Data Scientist')
('M Theory Solutions', 'Data Scientist / Predictive Modeler')
('eBay Inc.', 'Data Scientist/Applied Researcher')
('Accenture', 'Data Scientist Associate Principal')
('Metromile', 'Director of Analytics')
('eBay Inc.', 'Senior Data Scientist - GBH')
('Metromile', 'Senior Product Manager, Post-Purchase')
('eBay Inc.', 'Data Scientist 2')
Counter({'No': 519, 'Yes': 386})


In [86]:
Counter(df_test[(df_test['time'].notnull()) & (df_test['New']=="Yes")]['time'].tolist())

Counter({0.0: 6,
         1.0: 3,
         4.0: 10,
         5.0: 1,
         6.0: 4,
         7.0: 9,
         8.0: 19,
         9.0: 12,
         10.0: 26,
         11.0: 4,
         12.0: 3,
         13.0: 6,
         14.0: 9,
         15.0: 20,
         16.0: 19,
         17.0: 22,
         18.0: 7,
         20.0: 1,
         21.0: 14,
         22.0: 18,
         23.0: 6,
         24.0: 12,
         25.0: 5,
         27.0: 4,
         28.0: 9})